In [1]:
from src.linear_causality import LinearLKInformationFlow

In [2]:
import numpy as np
from tqdm import tqdm
# from causality_subspace import causality_subspace

dt=1
pn=1
T1=[];T2=[]
# pca = PCA(n_components=5)  # 假设我们想将数据降到2维  
Nt=10000;
np.random.seed = 5
# for i in tqdm(range(100)):
X1=np.random.randn(5,Nt)*0.2
X2=np.random.randn(5,Nt)*0.2

A1=np.array([[0.9,0.6,0,0.6,0],[0,0.8,0.6,0,0.6],[0,0,0.9,0,0],[0,0,0,0.2,0],[0,0,0,0,0.2]]).T;
A2=np.array([[0.2,0.6,0,0.6,0],[0,0.2,0.6,0,0.6],[0,0,0.2,0,0],[0,0,0,0.2,0],[0,0,0,0,0.2]]).T;

B1=-np.array([[0,0,0,0,0],[0,0.,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]]);
B2=-np.array([[0,0,0,0,0],[0,0.0,0,0,0],[0,0,0,0,0],[0,0,0,0.4,0],[0,0,0,0,0]]);
for it in range(1,Nt):
    random_noise_1 = np.random.randn(5)*0.1
    random_noise_2 = np.random.randn(5)*0.1
    X1[:,it]=A1@X1[:,it-1]+X1[:,it]+1*B1@X2[:,it-1] + random_noise_1
    X2[:,it]=A2@X2[:,it-1]+X2[:,it]+1*B2@X1[:,it-1] + random_noise_2
    
XX=np.zeros([10,Nt])
XX[:5]=X1
XX[5:]=X2

In [3]:
A1

array([[0.9, 0. , 0. , 0. , 0. ],
       [0.6, 0.8, 0. , 0. , 0. ],
       [0. , 0.6, 0.9, 0. , 0. ],
       [0.6, 0. , 0. , 0.2, 0. ],
       [0. , 0.6, 0. , 0. , 0.2]])

In [4]:
# segments = [(0, 1),(1,2),(2,3),(3,4),(4,5),(5,6),(6,7),(7,8),(8,9),(9,10)]
segments = [(0, 5),(5,10)]

In [5]:
lkif = LinearLKInformationFlow(np,1)
lkif.causality_estimate(XX[:,-10000:].T, segments)

In [7]:
result_dict = lkif.get_dict()

In [8]:
result_dict["information_flow"]

array([[-1.99110737e+00,  2.43419529e-03],
       [ 2.63149928e-01, -4.02687659e+00]])

In [9]:
result_dict["statistics"]["p99_critical_value"]

array([[0.03941324, 0.00364921],
       [0.01695025, 0.05020407]])

In [11]:
result_dict["statistics"]["p95_critical_value"]

array([[0.02998977, 0.0027767 ],
       [0.01289755, 0.03820058]])